In [8]:
import numpy as np
import pandas as pd
from astroquery.gaia import Gaia
from astropy.io import ascii
import emcee
import gala.potential as gp
import gala.dynamics as gd
import gala.integrate as gi
from gala.coordinates import reflex_correct
import astropy.coordinates as coord
import os

In [2]:
#define potantial 
potential = gp.MilkyWayPotential2022()
orbit_integrator = gi.LeapfrogIntegrator

In [3]:
#define constants
total_time = 5e6 #5 Myr
dt = -1.0

n_steps = int(total_time / abs(dt))


In [85]:
import numpy as np

def make_theta(star_params, cluster_params):
    # Convert each astropy Table column to a numpy array and take the first element.
    star_l   = np.array(star_params['l'])[0]
    star_b   = np.array(star_params['b'])[0]
    star_d   = np.array(star_params['distance_bj'])[0]
    star_pml = np.array(star_params['pm_l_poleski'])[0]
    star_pmb = np.array(star_params['pm_b_poleski'])[0]
    star_rv  = np.array(star_params["RV"])[0]
    
    cluster_l   = np.array(cluster_params['l'])[0]
    cluster_b   = np.array(cluster_params['b'])[0]
    cluster_d   = np.array(cluster_params['distance_bj'])[0]
    cluster_pml = np.array(cluster_params['pm_l_poleski'])[0]
    cluster_pmb = np.array(cluster_params['pm_b_poleski'])[0]
    cluster_rv  = np.array(cluster_params['RV'])[0]
    
    theta = [star_l, star_b, star_d, star_pml, star_pmb, star_rv,
             cluster_l, cluster_b, cluster_pml, cluster_pmb, cluster_rv]
    return theta

def make_star_stds(star_table):
    # Compute distance error and convert each astropy Table column to a numpy array, then take the first element.
    dist_err = np.array(star_table['distance_bj_high'])[0] - np.array(star_table['distance_bj'])[0]
    std_l   = np.array(star_table['l_err'])[0]
    std_b   = np.array(star_table['b_err'])[0]
    std_d   = dist_err
    std_pml = np.array(star_table['pm_l_err'])[0]
    std_pmb = np.array(star_table['pm_b_err'])[0]
    std_rv  = np.array(star_table['RV_err'])[0]

    star_stds = [std_l, std_b, std_d, std_pml, std_pmb, std_rv]
    return star_stds
   
def calc_cluster_radius(cluster_distance,angular_diameter):
    
    rad_diameter = angular_diameter* (np.pi/ (60*180)) # diamter of cluster in radians
    return cluster_distance*np.tan(rad_diameter/2)
    
def make_log_gauss(x,mu,sigma):
    return -0.5 * ((x - mu)/sigma)**2

In [42]:
def log_likelihood(theta, cluster_radius):
    '''theta  =['l','b','dist','pml','pmb','rv', 'cl_l', 'cl_b', 'cl_d', 'cl_pml', 'cl_pmb','cl_rv']
    log likelihood is the time seperation of the star and cluster after orbit integration
    
    calculate the orbit of the star and cluster in galactocentric
    
    '''
    star_l, star_b,star_d = theta[:3]
    star_pml, star_pmb, star_rv = theta[3:6]

    cluster_l, cluster_b, cluster_d = theta[6:9]
    cluster_pml, cluster_pmb, cluster_rv = theta[9:12]

    with coord.galactocentric_frame_defaults.set('v4.0'):
        galcen_frame = coord.Galactocentric()
    star_galactic_rep = coord.SkyCoord(l=star_l, b= star_b, pm_l_cosb= star_pml, pm_b = star_pmb, distance=star_d,
                                       radial_velocity = star_rv, frame='galactic')
    #convert to galactocentic
    star_galacto_rep = star_galactic_rep.transform_to(galcen_frame)
    star_galactic_rep = reflex_correct(star_galactic_rep) #correct for solar motion

    #same for the cluster
    cluster_galactic_rep = coord.SkyCoord(l=cluster_l, b= cluster_b, pm_l_cosb= cluster_pml, pm_b = cluster_pmb, distance=cluster_d,
                                       radial_velocity = cluster_rv, frame='galactic')
    cluster_galacto_rep =  cluster_galactic_rep.transform_to(galcen_frame)
    cluster_galactic_rep = reflex_correct(cluster_galacto_rep)
    #now integrate
    star_pos = gd.PhaseSpacePosition(star_galacto_rep.data)
    cluster_pos = gd.PhaseSpacePosition(cluster_galacto_rep.data)

    orbit_params = {"dt": dt, "n_steps": n_steps, "Integrator": orbit_integrator}
    star_orbit = potential.integrate_orbit(star_pos, **orbit_params)
    cluster_orbit = potential.integrate_orbit(cluster_pos, **orbit_params)

    time = star_orbit.t

    seperation = np.linalg.norm(star_orbit.xyz.to(u.pc) - cluster_orbit.xyz.to(u.pc), axis=0)
    min_sep = seperation[np.argmin(seperation)]

    if min_sep > cluster_radius:
        return -np.inf

    #to maximize liklehood
    return -time[min_sep]
    
def log__uniform_prior(theta):
    '''For now uniform prior'''
    star_pos = theta[:3]
    star_vel = theta[3:6]

    cluster_pos = theta[6:9]
    cluster_vel = theta[9:12]
    if (np.all(star_vel > 0) and np.all(cluster_vel > 0)):
        return 0.0  # Uniform prior
    return -np.inf

def log_normal_prior(theta,star_params,star_stds):
    '''Assume each parameter of the star comes from a normal distriubtion'''
    #these are sampled from walker
    star_l, star_b, star_d = theta[:3]
    star_pml, star_pmb, star_rv = theta[3:6]

    #true value of stars
    true_l, true_b, true_d= star_params['l'], star_params['b'], star_params['d']
    true_pml, true_pmb, true_rv = star_params['pm_l_poleski'], star_params['pm_b_poleski'], star_params["RV"]

    #true standard deviations 
    dist_err = star_params['distance_bj_high'] - star_params['distance_bj']
    std_l, std_b, std_d = star_stds['l_err'], star_stds['b_err'], dist_err
    std_pml, std_pmb, std_rv = stars_stds['pm_l_err'], stars_stds['pm_b_err'], stars_stds['RV_err']

    if star_d < 0.0 or np.sqrt(star_pml + star_pmb) >100: 
        return -np.inf
    log_l = make_log_gauss(star_l,true_l,std_l)
    log_b = make_log_gauss(star_b,true_b,std_b)
    log_d = make_log_gauss(star_d, true_d, std_d)
    
    log_pml = make_log_gauss(star_pml,true_pml, std_pml)
    log_pmb = make_log_gauss(star_pmb, true_pmb, std_pmb)
    log_rv = make_log_gauss(star_rv, true_rv, std_rv)
    return log_l + log_b + log_d, + log_pml + log_pmb + log_pmrv
    

In [43]:
def log_probability(theta,star_params, star_stds,cluster_radius):
    lp = log_normal_prior(tetha,star_params, star_stds)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta,cluster_radius)
    

# Load Data

In [38]:
cwd = os.getcwd()
home_files = os.path.dirname(cwd)
home_files = home_files + '/'
csv_files  = home_files + 'DATA/'
csv_files

HMXB_table = ascii.read(csv_files+'HMXB_20250310_.ecsv',format='ecsv')

# 1700-37 with NGC 6231

In [39]:
star_170037 = HMXB_table[HMXB_table['Name'] == '4U 1700-377']
star_170037

source_id,ra,ra_error,dec,dec_error,pmra,pmra_error,pmdec,pmdec_error,parallax,parallax_error,radial_velocity,radial_velocity_error,phot_g_mean_mag,l,b,ruwe,distance_bj,distance_bj_low,distance_bj_high,Name,Mx,Mx_err,Mo,Mo_err,RV,RV_err,Period,Period_err,Spin_period,Spin_period_err,distance_para,pm_l_poleski,pm_b_poleski,galactic distance,circular velocity,LSR velocity,mu_l_sol,mu_b_sol,RV_r_sol,mu_l_rot,mu_b_rot,RV_rot,Peculiar Velocity,Peculiar Radial Velocity,peculiar_mu_l,peculiar_mu_b,Peculiar Velocity 3D,dist_err,SpType,Mod_SpType,SpColor,pm_l_err,pm_b_err,l_err,b_err
,deg,mas,deg,mas,mas / yr,mas / yr,mas / yr,mas / yr,mas,mas,km / s,km / s,mag,deg,deg,,kpc,kpc,kpc,,,,,,km / s,km / s,,,,,kpc,mas / yr,mas / yr,kpc,km / s,km / s,mas / yr,mas / yr,km / s,mas / yr,mas / yr,km / s,km / s,km / s,mas / yr,mas / yr,km / s,kpc,,,,,,,
int64,float64,float32,float64,float32,float64,float32,float64,float32,float64,float32,float32,float32,float32,float64,float64,float32,float64,float64,float64,str23,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str21,str17,str10,float64,float64,float64,float64
5976382915813535232,255.9865659301791,0.021049444,-37.8441202722809,0.012138224,2.4137032635492046,0.02806476,5.021949969731823,0.021347089,0.6327358617075665,0.025948899,--,--,6.4204698,347.75444710988126,2.173492429913019,0.8081919,1.5139931999999998,1.4613417,1.5923752,4U 1700-377,1.96,0.19,46.0,5.0,-60.0,10.0,3.41166,4e-06,--,--,1.5804383163952436,5.459355496184255,1.114173464609666,6.679230942389423,237.63301645326374,-61.001772677836,-2.171203317603579,-1.0177427454204167,-7.952405001432037,-0.3724851740513693,0.05825101462333499,-9.02673557992658,59.32909375406844,-43.02085941864138,8.003043987839204,2.0736651954067478,73.2853035116697,0.06481477741223182,O6Iafcp,xkcd:blue,xkcd:blue,0.0240715261860172,0.012151228689122667,0.015476749868062746,0.006993492825535058


In [40]:
ngc6231 = ascii.read(csv_files+'NGC2631_param.ecsv',format='ecsv')
ngc6231_radius = calc_cluster_radius(ngc6231['distance_bj'],120)
ngc6231_radius[0]*1000


np.float64(27.075213015005065)

In [86]:
star_170037_theta = make_theta(star_170037, ngc6231)
star_170037_stds = make_star_stds(star_170037)

In [87]:
star_170037_theta

[np.float64(347.75444710988126),
 np.float64(2.173492429913019),
 np.float64(1.5139931999999998),
 np.float64(5.459355496184255),
 np.float64(1.114173464609666),
 np.float64(-60.0),
 np.float64(343.4762),
 np.float64(1.17),
 np.float64(-2.05709),
 np.float64(-0.929142),
 np.float64(-28.16)]

In [88]:
ndim = 12  # 3 position + 3 velocity for HMXB & Cluster
nwalkers = 24
#lbd pml pmb, vrad for star
#repat for cluster

#gagin for cluster
mean_vals = np.array([np.array(col) for col in star_170037_theta[0:3]])
std_vals = np.array([np.array(col) for col in star_170037_stds[0:3]])

mean_star_vals = mean_vals.flatten()
std_vals = std_vals.flatten()

initial_pos = np.hstack([
    np.random.normal(mean_vals, std_vals, size=(nwalkers, 3)),
])
initial_pos = np.random.normal(mean_vals, std_vals, size=(nwalkers, 3))


In [89]:
# initial_hmxb_pos = np.random.normal(hmxb_mean_pos, hmxb_std_pos, (nwalkers, 3))
# initial_hmxb_vel = np.random.normal(hmxb_mean_vel, hmxb_std_vel, (nwalkers, 3))
# initial_cluster_pos = np.random.normal(cluster_mean_pos, cluster_std_pos, (nwalkers, 3))
# initial_cluster_vel = np.random.normal(cluster_mean_vel, cluster_std_vel, (nwalkers, 3))

# # Combine these into a single (nwalkers x ndim) array
# initial_pos = np.hstack([
#     initial_hmxb_pos, 
#     initial_hmxb_vel, 
#     initial_cluster_pos, 
#     initial_cluster_vel
# ])

NameError: name 'hmxb_mean_pos' is not defined

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(cluster_radius,))
state = sampler.run_mcmc(initial_pos, 1000, progress=True)

# Extract results
samples = sampler.get_chain(discard=500, thin=10, flat=True)
kinematic_ages = -samples[:, 0]  # Convert back from negative log-likelihood

kinematic_age_median = np.median(kinematic_ages)
kinematic_age_std = np.std(kinematic_ages)
